## Imports

In [ ]:
# standard libs
import os
import pandas as pd
import numpy as np

print(os.getcwd())

## Data

### Import Data

In [ ]:
# import cities response df
cities21_df = pd.read_csv("../input/cdp2021/2021_Cities_Water_Security_Risks.csv")
cities20_df = pd.read_csv("../input/cdp-unlocking-climate-solutions/Cities/Cities Responses/2020_Full_Cities_Dataset.csv")
cities19_df = pd.read_csv("../input/cdp-unlocking-climate-solutions/Cities/Cities Responses/2019_Full_Cities_Dataset.csv")



### Data Cleaning & EDA



In [ ]:
risk21=cities21_df.rename(columns={'Account Name': 'City'})
risk21.dropna(subset=['Water security risk drivers'],inplace=True)
risk21.drop_duplicates(inplace=True)

cities20_df = cities20_df[cities20_df['Question Number'] == '14.2a']\
    .rename(columns={'Organization': 'City'})
cities20_df.dropna(subset=['Response Answer'],inplace=True)
risk20=cities20_df.pivot_table(index=['Account Number', 'City', 'Row Number'],
                                     columns='Column Name', 
                                     values='Response Answer',
                                     aggfunc=lambda x: ' '.join(x)).reset_index()
risk20.drop_duplicates(inplace=True)

cities19_df = cities19_df[
    cities19_df['Question Name'] == 
    'Please identify the risks to your city’s water supply as well as the timescale and level of risk.']\
    .rename(columns={'Organization': 'City'})
cities19_df.dropna(subset=['Response Answer'],inplace=True)
risk19=cities19_df.pivot_table(index=['Account Number', 'City', 'Row Number'],
                                     columns='Column Name', 
                                     values='Response Answer',
                                     aggfunc=lambda x: ' '.join(x)).reset_index()
risk19['Risks'][(risk19['Risks']=='Inadequate or ageing infrastructure')]='Inadequate or ageing water supply infrastructure'
risk19=risk19.rename(columns={'Risks':'Water security risk drivers'})
risk19.drop_duplicates(inplace=True)
risk21.head()

In [ ]:
risk20.head()

In [ ]:
risk19.head()

In [ ]:
def countrisk(df):
    df1=pd.crosstab(index=df['Water security risk drivers'], columns='count').sort_values(by='count',ascending=False)
    if sum(df1.index.isin(['Question not applicable']))>0:df1.drop('Question not applicable',inplace=True)
    df1=df1[df1['count']>5]
    return(df1)

In [ ]:
df21=countrisk(risk21)
df21.plot.pie(y='count',figsize=(6,6),legend=False)

In [ ]:
df20=countrisk(risk20)
df20.plot.pie(y='count',figsize=(6,6),legend=False)

In [ ]:
df19=countrisk(risk19)
df19.plot.pie(y='count',figsize=(6,6),legend=False)

In [ ]:
df21=df21.rename(columns={'count': 'count21'})
df20=df20.rename(columns={'count': 'count20'})
df19=df19.rename(columns={'count': 'count19'})
outer_merged = pd.merge(df21, df20, how="outer",on='Water security risk drivers')
outer_merged = pd.merge(outer_merged , df19, how="outer",on='Water security risk drivers')
outer_merged

In [ ]:
idx=risk21['Water security risk drivers'].isin(outer_merged.index[0:5].tolist())
risk5_2021=risk21[idx][['CDP Region','City','Water security risk drivers']].drop_duplicates()
risk5=risk5_2021.pivot_table(index=['CDP Region'],columns='Water security risk drivers',values='City',
                             aggfunc=lambda x: len(x),margins=True).sort_values(by='All',ascending=True)
risk5

In [ ]:
risk5.iloc[:,0:5].plot.barh( stacked=True,
        title='Stacked Bar Graph by Region')